# A more general note on feature engineering: What is feature engineering?

Here I will say a bit about feature engineering. It is basically just manipulation of our features, but I will try to get a bit into the logic behind it in this note book. Given that There is no specific text on feature engeering on the curriculum, this notbook is a bit tesst heavy - please di try to read through it. Furthermore, what I do here is mostly for illustrative purporse. A millon other thinks could have be done, but the exercises below should be enough to get you and intuition of why we do it and who we do it. Best of luck. 

So, a while ago we chose some features as "features of interest". Specifically: `nlights_mean`, `pop_gpw_sum`, `agri_ih`, `barren_ih`, `urban_ih`, `petroleum_y` and `excluded`. We could have choosen more, but this seemed like a prudent little roster for the sake of learning. Now, lets call these features our *raw* features; meaning that we have not changed anyhthing about them (except of cause the interpolation...). Creating new features from such raw features is called *feature engineering*. As such our creation of 'ratio_best' and 'binary_best' can also be seen as feature engineering.

In Machine Learning feature engineering entails manipulating the data to maximise its predictive power. That is, some alternative representation of the raw features might be better at predicting the target since this representation might better emulate the data generating process. Lets take $voting$ (y) and $age$ (x) as an exemple. if we wanted to predict the expected voting percentage at some election it is well know that $age**2$ is a better predictor than just $age$. For instance, in python we would thus create a new feature (age2) like so:

`df['age2'] = np.sqrt(df[age])`

There are theoritical grounds explianing why $age^2$ is better at predicting voting than $age$, but in conventional Machine Learning the merit of the feature is strictly an empirical one: Does it increase prediction power? If so, good, we'll take it.

In conventional Political Science we also do feature engineering - though often under some different moniker. Here the goal is to manipulating the data to more accurately capture the theoretical mechanism mapping $x$ to $y$. Thus, the point of changing $age$ to $age^2$ is here not to increase prediction power - since prediction is rarely the goal - instead the point would be to adequitly capture and estimate the relationship between $age$ and $voting$. Naturally, one could see whether $age^2$ is more significant than $age$ and so on, but as you might have read in Ward et al. (2010) or Schrodt (2014) this might not be very prudent. Thus the theoritical justification is paramount: the older people are, the higher the probability they will vote - but only until the start getting elderly and the probability of voting starts decreasing again (wheter this is a generational thing or a aging thing is another matter). Returning to the field of conflict studies we might want to change a wealth measure from absolute to relative if we believes relative deprivation to be more important than greed or state capacity.

Now, this is a course about *Machine Learning in Conflict Prediciton*, so our goal is prediction power, not estimating the correct and unbiased relationship between phenomenons. However, theory might still be our friend! As illustrated in the exemple above, $age^2$ is both theoritically prudent and a good predictor. This should come as no surprice: teoritically sound features should bring os closer to the data generting process, wich inturn will increase the prediction power of our model (See: Hegre et al., 2017a; Ward and Beger, 2017; Colaresi and Mahmood, 2017; Schneider et al., 2017; McElreath, 2018). Thus, thinking about theory and data generating processes is a very good place to start when we du feature engineering. After all we can manipulate our 'raw' features in infinatly many ways, so some guidience is highly appriciated. 

It is, however, important to remember that we are still just looking to increase prediction power and are not out to prove points regarding the actual data generation process. We might be able to do that with prediction (see McElreath 2018) but it is not something that is covered in this course. For instance, we will not conclude anything about the true underlying data generating process just because some feature is better as a predictor then another: Say if an relative wealth measure is a better predictor than an absolute wealth measure. In other words, you might use thoery to create better predictors, but be careful if you want to use predictions to challange, validate or support theory in Social Science as this is still a very controversial topic. 

Notably, however, this is standart in most Natural Sciences.. On to the exercise!

# 4A.0:

Load your libraries numpy, pandas, geopandas, matplotlib.pyplot, pickle and time. This time also import seaborn as sns.

Load in your train_set.pkl - we will not need val_set ot test_set yet.

Create a object `years` with all unique years (`train_set['year'].unique()`) in you train set. This is not super important, but it will be nice to have along the way.

Try to print all your features and a count of them now:

In [ ]:
print(len(train_set.columns))
print(train_set.columns)

Now, do some of your favorite checks, to insure that everything is in order:

# 4A.1 Past Violence

We will now create some features that aims to capture the conflict history of the individual cells. We want such features since it is well known that conflict in one geographic location is dependent on the conflict history of said geographical location (Beck et al., 1998, 1263-1264). This phenomenon is often referred to as the conflict trap (Hegre et al., 2017). It has been shown that the probability of conflict occurring/reoccurring/continuing in a specific country can increase with up to 20% the first year after a conflict event. Hereafter the impact of said event decreases each year but might still be observed +25 years later (Hegre et al., 2017, 249). How severe a specific trap will be is partly a function of the magnitude of conflict; more conflict, longer trap (Hegre et al., 2017, 255). 

Unsurpricingly, modelling the conflict trap or any form aof temporal dependency in conflict can be done in infinately many ways. And indeed it almost have been. We will start with something very simple. Cederman et al. (2013) just count the number of previous conflicts in the geographical unit of analysis. We will start by doing something very alike.

Specifically, we can start by creating a feature denoting the cumulative sum (`cumsum`) of all past fatalities (`best`), the sum of past conflcit magnitude (`log_best`) and the sum of all past conflict events (`binary_best`). For prudence I like to sort by `year` before I do these kinds of operations. The data should already be sorted but better safe than sorry. Here you want to groupby `gid`

In [ ]:
train_set['past_fatalities'] = train_set.sort_values('...').groupby(['...'])['...'].cumsum()
train_set['past_magnitude'] = train_set.sort_values('...').groupby(['...'])['...'].cumsum()
train_set['past_events'] = train_set.sort_values('...').groupby(['...'])['...'].cumsum()

Lets do som checking (waht ever you feel would reveal any errors):

Looks good? Hopefully so. However, this was on gid (prio grid cell) level, but country level could also be relevant. Code is the same as the one above, but groupby should be `gwno` which is the country specific number

In [ ]:
train_set['past_fatalities_country'] = train_set.sort_values('year').groupby(['...'])['best'].cumsum()
train_set['past_magnitude_country'] = train_set.sort_values('year').groupby(['...'])['log_best'].cumsum()
train_set['past_events_country'] = train_set.sort_values('year').groupby(['...'])['binary_best'].cumsum()

Try making a blot or two to see if you have succesfully created country level features:

Now some countries are much larger then others so it might be prudent to normalize be the area of the country. and population. But, before we start dividing one feature with another, it is prudent to nomalize or standanise the features so the scale is somewaht similar. For instance, if the numerator get to small (relativly spaeking) the fraction will be very small and do to nummerical underflow we will just get zero. If denominator get gets to small (relativly speaking) numerical underflow lead to division by zero which will give us an error. 

I will use alot of nomalization below, you can read about the difference between normalization and standartization here: https://medium.com/@rrfd/standardize-or-normalize-examples-in-python-e3f174b65dfc. They use a sklearn implimentation, but I will just stick with good old math.

(no need to change anything here)

In [ ]:
features_to_normalize = ['gwarea', 'interp_pop_gpw_sum', 'past_fatalities_country', 'past_magnitude_country', 'past_events_country']

for feature in features_to_normalize:
    
    new_name = 'norm_' + feature
    train_set[new_name] = (train_set[feature]- train_set[feature].min())/(train_set[feature].max()-train_set[feature].min())
    

Now, divide the three features with `train_set['norm_gwarea']+1`:

In [ ]:
train_set['past_fatalities_country_Narea'] = train_set['norm_past_fatalities_country'] / (train_set['norm_gwarea']+1)
train_set['past_magnitude_country_Narea'] = train_set['norm_past_magnitude_country'] / (train_set['norm_gwarea']+1)
train_set['past_events_country_Narea'] = train_set['norm_past_events_country'] / (train_set['norm_gwarea']+1)

**Why +1 in the end here?**

Do some plotting and checking:

**What would be some theoritical justifiation for the cell (gid) level features?**    
**What would be some theoritical justifiation for the country (gwno) level features?**

Now, try to divide with `(train_set['norm_interp_pop_gpw_sum']+1)` instead:

In [ ]:
# appears most intiative when plotted
train_set['past_fatalities_country_Npop'] = train_set['norm_past_fatalities_country'] / ...
train_set['past_magnitude_country_Npop'] = train_set['norm_past_magnitude_country'] / ...
train_set['past_events_country_Npop'] = train_set['norm_past_events_country'] / ...

And a plot or two

**What - theoretically speaking - do you think is the difference between dividing by population vs. area?**   
**Wich seems most prudent?**

Now theoritically it does seem a bit weird if these measures just keep accumulating. Collier and Hoeffler (2004) use a linear decay function counting years since last conflict and Hegre and Sambanis (2006) use a linear decay function counting years since last peace. We however, will create somthing similar to Parrys (2010) deterioration index.

# 4A.2 Parrys index

A deterioration index recently proposed by Perry (2013) will serve as an initial example. Perry’s idea is also related to the phenomenon of the conflict trap and thus the notion of some inertia in conflict. However Perry specifies that to mimic the conflict trap such index might deteriorate over time. To create Parry’s time deteriorating index we would start by including the number of fatalities in some geographic unit for each of the last ten years as features. The deterioration rate is then incorporated by dividing these fatalities with the number of years passed since the corresponding events. Thus, a feature pertaining to fatalities two years ago will have, as its values, half of the fatalities observed that year (Perry, 2013, 14). In short the proposed index aims to fit the insight that the closer in time a geographic region is to pastconflicts, the more likely said region is to experience conflict again.

We will not impliment the exact index proposed by Perry, but somehting similar. We'll create an exponential deterioration index wich runs over 5 years (`best_decay5`) and one that like Perry's runs over 10 years (`best_decay10`). No need to change in the code below - or worry to much about it - just run it. It will take some time though.


In [ ]:
def decay5(data, window=5):

    alpha = 2 /(window + 1.0)
    alpha_rev = 1-alpha
    n = data.shape[0]

    pows = alpha_rev**(np.arange(n+1))

    scale_arr = 1/pows[:-1]
    offset = data.iloc[0]*pows[1:]
    pw0 = alpha*alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums*scale_arr[::-1]
    return out

def decay10(data, window=10):

    alpha = 2 /(window + 1.0)
    alpha_rev = 1-alpha
    n = data.shape[0]

    pows = alpha_rev**(np.arange(n+1))

    scale_arr = 1/pows[:-1]
    pw0 = alpha * alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = cumsums * scale_arr[::-1]
    return out


In [ ]:
train_set['best_decay5'] = train_set.sort_values('year').groupby(['gid'])['best'].apply(decay5)
train_set['best_decay10'] = train_set.sort_values('year').groupby(['gid'])['best'].apply(decay10)

The plot below might help give some intuition what is going on here. No need to change any code. Try and run it (a couple of times even. Should be a new one each time):

In [ ]:
some_gids = train_set[train_set['ratio_best']>0.6]['gid'].sample(10, replace = False)
colors = sns.color_palette("hls", some_gids.shape[0])

plt.figure(figsize = (15,5))

for i, gid in enumerate(some_gids):
    
    plt.plot(years, train_set[train_set['gid'] == gid]['best_decay5'], '--', color  = colors[i])
    plt.plot(years, train_set[train_set['gid'] == gid]['best_decay10'], '-', color  = colors[i])
    
    plt.plot(years, train_set[train_set['gid'] == gid]['best'], 'o', color  = colors[i])
    
plt.show()


We also see that this index provides some regularisation.

**What do I here mean by regularization**
**How does best_decay5 (dashed) differ from best_decay10 (solid)?**

One of these (or both in some combination) might serve as a representation of the phenomenon of the conflict trap. However, they do appear as somewhat ad hoc  solutions. There is no reason to cap the effort at five or ten years and there is no theoretical or practical reason to choose neither Perrys suggested deterioration rate or my exponential decay. The deterioration rate might have an altogether different functional form. The point is that if we do not know the relevant functions, efforts should been made to estimate them rather than assume them. That however, is a challange for another time.

# 4A.3 Wealth and State Capacities

Easily one of the most robust findings in country level studies of intra state conflicts is that GDP per capita (Often logged and adjusted for purchasing power parity (ppp)) has a negative effect on the probability of civil war onset (Collier and Hoeffler, 1998; Fearon and Laitin, 2003; Collier and Hoeffler, 2004; Hegre and Sambanis, 2006; Blattman and Miguel, 2010) and also to some extent the conflict duration (Fearon, 2004; Hegre et al., 2009).

A number of mechanisms have been proposed linking GDP to conflict, Two have been especial prolific. The first is championed by Collier and Hoeffler (1998, 2004) who sees GDP per capita as a proxy for opportunity-cost. That is what a given citizen have to loss by engaging in conflict. The second story draws on insight from Skocpol  (1979) and also echoes the gospel of modernization theory (Lipset, 1959). Here GDP per capita is seen as as proxy for state capacities. Simple put; weak or fragile states have low GDP per capita and these states are more conflict prone (Fearon and Laitin, 2003, 88).
 
A measure for GCP (Gross cell product) per capita (ppp) is included in the PRIO GRID from the Gecon dataset (Nordhaus, 2006) and this measure could be aggregated  creating a feature mimicking GDP per capita (ppp) (Tollefsen and Buhaug, 2015). However the data available from the directly from PRIO only extent to 2010 and could be very prone to measurement errors in remote regions. Fortunately Elvidge et al. (2009) and Chenand Nordhaus (2011) have shown that Night light emission can serve as an proxy for economic activities - especially for countries and areas with low-quality statistical systems and few or no recent population or economic censuses (Chen and Nordhaus, 2011). Indeed using night light emission have bin explicitly proposed as a proxy for wealth by Cederman et al. (2013, p.  101) in context of conflict studies.

Again, we could makle infinitely many different features using the raw feature `interp_nlights_mean` - in it self of in combinaiton with other of our raw features. However, we shall try to constrain ourselves and settle for a smaller roster.

A first thing wich seems prudent is to make an emmssion per capita feature by dividing night light emission (`train_set['interp_nlights_mean']`) by population (`train_set['norm_interp_pop_gpw_sum']`):

$$\frac{Nlights}{normalized\_Pop +1}$$


In [ ]:

train_set['cell_light_Pcap'] = .../...


Fell free to do some plotting for reference:

Also try to run the plot below:

In [ ]:
onekgid = train_set['gid'].sample(1000)

plt.scatter(x = train_set[train_set['gid'].isin(onekgid)]['interp_nlights_mean'], 
            y = train_set[train_set['gid'].isin(onekgid)]['cell_light_Pcap'], 
            c = train_set[train_set['gid'].isin(onekgid)]['interp_pop_gpw_sum'])
plt.show()

Do you understand what you see here? A very close relationship, but we do see in the plot that cells with very high population gets panalized somewhat on in the new feature, wich is what we wanted to do.

Of course, we would like a country wide measure along our cell specific measure. Not least since the effect found in the litterature peatains to state wealth and not local wealth. It is tempting to just take the country mean, but that would make large but sparsely populated countries such as Canada and Australia very "poor". As such we will sum the country `interp_nlights_mean` and divide by normalized population (`norm_interp_pop_gpw_sum`):

This will be proxy a country wealth per capita measure. But first we need the individual measures. Sort values according to `year`, group by `gwno`

In [ ]:
train_set['interp_nlights_mean_country'] = train_set.sort_values('...').groupby(['...'])['interp_nlights_mean'].transform(np.sum)
train_set['norm_interp_pop_gpw_sum_country'] = train_set.sort_values('...').groupby(['...'])['norm_interp_pop_gpw_sum'].transform(np.sum)#

Do some plotting:

Now, divide the two new features: `(train_set['interp_nlights_mean_country'])/(train_set['norm_interp_pop_gpw_sum_country']+1)` to create `train_set['country_light_Pcap'] `

In [ ]:
train_set['country_light_Pcap'] = ...

Plot it - always plot it:

And do the same with area, sp divide the two features: `(train_set['interp_nlights_mean_country'])/(train_set['norm_gwarea']+1)` to create `train_set['country_light_Area'] `

In [ ]:
train_set['country_light_Area'] = .../...

Naturrally, it migth also be that is not som absolute measure of wealth/state capacity that is the moset salinet prediciton meby it is something more relative.

# 4A.3 low and high ratio from Cederman, Gleditsch and Buhuag

Lets talk about inequality and deprivation. If we leave the "Strong State" proposition behind and focus on the satisfaction of the individual citizens it is only  natural to argues this satisfaction should be considered a function of what said citizin have and what she believe she rightfully should have.  This is the crux of Robert Gurr’s (1970) Relative Deprivation Theory. Perhaps one of the most seminal takes on inequality and conflict, Gurr (1970) defines relative deprivation:

*"\[...\] as actor's perception of discrepancy between their value expectation and their value capabilities. Values expectation are the goods and conditions of life to which people believe they are rightfully entitled. Value capabilities are the goods and conditions they think they are capable of getting and keeping." (Gurr 1970, 24).*

While intuitively appalling, Gurr’s theory was awarded little credit doing the haydays of comparative cross country conflict studies. Supporting statistical results failed to materialize. However,  Cederman  and  Gleditsch  (2009);  Cederman  et  al.  (2013) have noted that the aggregated country level features conventional used as indicators for inequality might lead to mis-specifications; that is, they do not properly measure the theoretical concept of relative deprivationor the correct mechanisms through which inequality affects conflict-propensity (Cederman et al.,2013). Acknowledging this critique we can utilized the operationalization put forth in (Cedermanet al., 2013, p.  104-105)

$$y_g = \textrm{country year mean}\quad ,\quad  y_c = \textrm{cell year value}$$
$$\textrm{low_ratio} = y_c/y_g  \quad \textrm{if} \quad y_c < y_g, \quad 1 \quad \textrm{otherwise}$$

Thus the feature takes the value 1 if the cell is below the rest of the country. As before the featureuses on Night Light emission (cell-year mean) as indicator for wealth (no need to change any code here).

In [ ]:

train_set['interp_nlights_mean_norm'] = (train_set['interp_nlights_mean'] - train_set['interp_nlights_mean'].min())/(train_set['interp_nlights_mean'].max()-train_set['interp_nlights_mean'].min())

train_set['interp_nlights_mean_country_norm'] = (train_set['interp_nlights_mean_country'] - train_set['interp_nlights_mean_country'].min())/(train_set['interp_nlights_mean_country'].max()-train_set['interp_nlights_mean_country'].min())
train_set['country_light_Pcap_norm'] = (train_set['country_light_Pcap'] - train_set['country_light_Pcap'].min())/(train_set['country_light_Pcap'].max()-train_set['country_light_Pcap'].min())
train_set['country_light_Area_norm'] = (train_set['country_light_Area'] - train_set['country_light_Area'].min())/(train_set['country_light_Area'].max()-train_set['country_light_Area'].min())

train_set['low_ratio_light'] = np.minimum(train_set['interp_nlights_mean_country_norm'] / train_set['interp_nlights_mean_norm'],1)  
train_set['low_ratio_light_Pcap'] = np.minimum(train_set['country_light_Pcap_norm'] / train_set['interp_nlights_mean_norm'],1)  
train_set['low_ratio_light_Area'] = np.minimum(train_set['country_light_Area_norm'] / train_set['interp_nlights_mean_norm'],1)


In [ ]:
plt.hist(train_set['interp_nlights_mean_norm'], alpha = 0.4)
plt.hist(train_set['country_light_Pcap_norm'], alpha = 0.4)
plt.hist(train_set['low_ratio_light'], alpha = 0.4)
plt.hist(train_set['low_ratio_light_Pcap'], alpha = 0.4)
plt.hist(train_set['low_ratio_light_Area'], alpha = 0.4)

plt.show()

Plot these new features:

Now, these relative deprevation features could be done in infinatly many ways, and I not saying these are the best conceptualiations or implimentaions. The point is simply illustrating the posibilities and give you some food for thourgh.

# 4A.4 Ethnicity and Exclusion

As with inequality the link between ethnic diverse societies and conflict propensity have been ridden with disagreement and controversies. In the quantitative literature results have remained somewhat inconsistent (Blattman  and  Miguel,  2010,  23-24).  A  number of studies have fund different - and sometime quite  convoluted - relationships between ethnicity or discriminationand conflict (Collier and Hoeffler, 1998; Fearon, 2004; Blimes, 2006; Hegre and Sambanis, 2006;Goldstone et al., 2010), while other studies have fund little or no trace of the connection (Fearonand Laitin, 2003; Collier and Hoeffler, 2004).

As with the problem with inequality the lack of discernible results have often been attributed to poor feature specifications, a framework not capturing the proposed theoretical mechanism and not least country aggregated data (Blimes, 2006; Blattman and Miguel, 2010; Cederman et al.,2013). Mirroring their effort concerning  inequality Cederman et al. (2013) have used new desegregated data (Girardin et al., 2015) to closely model the theoretical mechanism proposed. Without going to much in-depth here the feature Cederman et al. (2013) utilizes aims to capture the effect of *horizontal inequalitiesas* defined by (Cederman et al., 2013, 31-35). That is the  systematic discrimination or political exclusion of and coherent ethnic group. Thou not framed in the theoretical context of horizontal inequalities Goldstone et al. (2010) finds results supporting the argument using the Minority at Risk data from Gurr (1995).

In our data the feature `excluded` denotes the number of excluded ethnic groups (e.i.  discriminated or powerless) in a given cellat a given year.  The measures are originally from the GeoEPR/EPR data by Vogt et al. (2015). To suit the theoretical argumentation laid forth in Cederman et al. (2013)(side) (Tollefsenand Buhaug, 2015).  lets create a dummy (excluded_binary) which simply denoteifthere are any excluded ethnic groups.

Try binarizing `train_set['interp_excluded']` so everything above 0 becomes 1. (And remember the \*1 hack that turns booleans to intengers)

In [ ]:
train_set['interp_excluded_binary'] = ...

Plot your creation:

So why did I do that and not somthing else? Why did I leave out that cleaver thing you are thinking about right now? Exactly, the possibilities are endless, and thus we need theory on our side, or some algorithm that can learn its own features. Tree-models do that to some extent and as I will talk about in the end of the semester Atirficial Neuaral Nets are the king of the hill here. That does not mean we don't have to think. It just meand we can consentrate on model construction and not feature engeneering, which as you can see above, can be both time consurming and fustrating..

Try to print all your features and a count of them now:

**How many features du you have compared to when you started this notebook?**

# 4A.5 Optional

So, next time we will create all the same features for the validation set before we test wich features holds the most prediction power. If you are up for it you can start now! 

For many of the features you can just run the code from above on your validation set (`val_set`) and you should be fine. For other features - which uses information across time such as the conflcit trap features - you should just take the last observed value (2012) from a given cell (`gid`) and transfere that to the corresponding cell (`val`) in the validation set (`val_set`). I do something very similar in the `week3correction` notebook. 